In [ ]:
# -*- coding: utf-8 -*-
#
# splines.py - Spline Interpolation to Fit Curves.
#
#
# Autor: Pedro Garcia Freitas [sawp@sawp.com.br]
# License: Creative Commons
#      <http://creativecommons.org/licenses/by-nc-nd/2.5/br/>
#
# Feb 2011
#
from math import sqrt
from copy import deepcopy
from itertools import product


def thomas(A, b):
    """
    Return a list with the solution of linear system.

    thomas(A, b)

    INPUT:
    * A: a matrix of coefficients
    * b: a list, relative a vector of independent terms

    return: a list with unknown vector 'x' of system A*x = b.

    Author: Pedro Garcia <sawp@sawp.com.br>
    see: http://www.sawp.com.br

    License: Creative Commons
             http://creativecommons.org/licenses/by-nc-nd/2.5/br/

    Sep 2010
    """
    n = len(b)
    terms = deepcopy(b)
    x = []
    d = []
    upper = []
    lower = []

    for i in xrange(n):
        d.append(A[i][i])
        x.append(0.0)
        upper.append(0.0)
        lower.append(0.0)
    for i in xrange(n - 1):
        upper[i] = A[i][i + 1]
        lower[i] = A[i + 1][i]

    for i in xrange(1, n):
        d[i] = d[i] - lower[i - 1] * upper[i - 1] / d[i - 1]
        terms[i] = terms[i] - lower[i - 1] * terms[i - 1] / d[i - 1]
    x[n - 1] = terms[n - 1] / d[n - 1]

    # (regressive replacement)
    for i in xrange(n - 2, -1, -1):
        x[i] = (terms[i] - upper[i] * x[i + 1]) / d[i]
    return x


def tridiagonal_solve(d, e, c, b):
    nr = len(b) - 1
    n = xrange(nr + 1)
    A = [[0.0 for i in n] for j in n]
    n = xrange(1, nr)
    A[0][0] = d[0]
    A[0][1] = e[0]
    A[1][0] = c[0]
    for i in n:
        A[i][i] = d[i]
        A[i - 1][i] = c[i]
        A[i + 1][i] = e[i]
    A[nr - 1][nr] = e[nr]
    A[nr][nr] = d[nr]
    return thomas(A, b)


def spline(x, x_j, f_j, n=3):
    """
    Return x evaluated in Spline interpolation function.

    spline(x, x_j, f_j, n=3)

    INPUT:
      * x: float, evalue at this point
      * x_j: LIST, tabular points
      * f_j: LIST, tabular points (must be same size of x_j)
      * n: spline degree (1=linear,2=quadratic,3=cubic(default),4= quartic)

    return:
      * f(x): interpolated and evaluated x value

    Author: Pedro Garcia [sawp@sawp.com.br]
    see: http://www.sawp.com.br

    License: Creative Commons
             http://creativecommons.org/licenses/by-nc-nd/2.5/br/

    Feb 2011
    """

    def cubic_spline(xi, fi):
        """ Fit a curve using a cubic spline"""
        n = len(xi) - 1
        h = []
        g = []
        d = []
        b = []
        c = []
        # diferences between points
        for i in xrange(n):
            hi = xi[i + 1] - xi[i]
            gi = fi[i + 1] - fi[i]
            h.append(hi)
            g.append(gi)
        # generate 3-diagonal matrix
        for i in xrange(n-1):
            di = 2 * (h[i + 1] + h[i])
            bi = 6 * (g[i + 1] / h[i + 1] - g[i] / h[i])
            ci = h[i + 1]
            d.append(di)
            b.append(bi)
            c.append(ci)
        # solve the tridiagonal system
        g = tridiagonal_solve(d, c, c, b)
        # solution
        pol = [0.0 for i in xrange(n)]
        pol[n-1] = 0.0
        for i in xrange(1, n):
            pol[i] = g[i-1]
        return pol

    if type(x_j) != type(f_j) or type(x_j) != type([]):
        print "Error: wrong type parameters"
        return float("NaN")

    if len(x_j) != len(f_j):
        print "Error: the tabular points must have same size"
        return float("NaN")

    if n == 1:
        pass
    elif n == 2:
        pass
    elif n == 4:
        pass
    else:
        polinomial = cubic_spline(x_j, f_j)

    # this is just to assure convergence
    m = 100
    for i in xrange(m):
        k = 1
        dx = x - x_j[0]
        while dx >= 0:
            k = k + 1
            dx = x - x_j[k]
        k = k - 1
        # encontra um valor para a funcao f(x)
        dx = x_j[k + 1] - x_j[k]
        alpha = polinomial[k + 1] / (6 * dx)
        beta = - polinomial[k] / (6 * dx)
        gamma = f_j[k + 1] / dx - dx * polinomial[k + 1] / 6
        ro = dx * polinomial[k] / 6 - f_j[k] / dx
        f = alpha * (x - x_j[k]) ** 3 + \
            beta * (x - x_j[k + 1]) ** 3 + \
            gamma * (x - x_j[k]) + \
            ro * (x - x_j[k + 1])
    return f

if __name__ == "__main__":
    f = [0.0]
    f.append(0.4400505857)
    f.append(0.5767248078)
    f.append(0.3390589585)
    f.append(-0.0660433280)
    f.append(-0.3275791376)
    f.append(-0.2766838581)
    f.append(-0.0046828235)
    f.append(0.2346363469)
    f.append(0.2453117866)
    f.append(0.0434727462)


    termos = []
    for i in xrange(len(f)):
        termos.append(float(i))

    # test interpolation
    for i in xrange(1, 18):
        print "J[", i * 0.5, "] ", spline(i * 0.5, termos, f, 3)